# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
S. Vaughan  ->  S. Vaughan  |  ['S. Vaughan']


A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 88 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.13976


extracting tarball to tmp_2510.13976...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.14083
extracting tarball to tmp_2510.14083...

 done.


...

/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2510.14529/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/results' from 'tmp_2510.14529/tables/results.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'tables/sample_table' from 'tmp_2510.14529/tables/sample_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python

extracting tarball to tmp_2510.14779... done.


Found 176 bibliographic references in tmp_2510.14779/main.bbl.
Error retrieving bib data for Nestor-Shachar:2023aa: 'author'
Error retrieving bib data for de-Graaff:2024ab: 'author'
Error retrieving bib data for Nestor-Shachar:2023aa: 'author'
Error retrieving bib data for Nestor-Shachar:2023aa: 'author'
Retrieving document from  https://arxiv.org/e-print/2510.14839


extracting tarball to tmp_2510.14839...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.14083-b31b1b.svg)](https://arxiv.org/abs/2510.14083) | **NGTS-11 c: a transiting Neptune-mass planet interior to the warm Saturn NGTS-11 b**  |
|| D. R. Anderson, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-10-17*|
|*Comments*| *12 pages, 6 figures, submitted to ApJ*|
|**Abstract**|            We report the discovery of NGTS-11 c, a transiting warm Neptune ($P \approx 12.8$ d; $M_{p} = 1.2^{+0.3}_{-0.2} M_{\mathrm{Nep}}$; $R_{p} = 1.24 \pm 0.03 R_{\mathrm{Nep}}$), in an orbit interior to the previously reported transiting warm Saturn NGTS-11 b ($P \approx 35.5$ d). We also find evidence of a third outer companion orbiting the K-dwarf NGTS-11. We first detected transits of NGTS-11 c in TESS light curves and confirmed them with follow-up transits from NGTS and many other ground-based facilities. Radial-velocity monitoring with the HARPS and FEROS spectrographs revealed the mass of NGTS-11 c and provides evidence for a long-period companion ($P > 2300$ d; $M_{p} \sin i > 3.6 M_{\mathrm{Jup}}$). Taking into account the two additional bodies in our expanded datasets, we find that the mass of NGTS-11 b ($M_{p} = 0.63 \pm 0.09 M_{\mathrm{Sat}}$; $R_{p} = 0.97 \pm 0.02 R_{\mathrm{Sat}}$) is lower than previously reported ($M_{p} = 1.2 \pm 0.3 M_{\mathrm{Sat}}$). Given their near-circular and compact orbits, NGTS-11 c and b are unlikely to have reached their present locations via high-eccentricity migration. Instead, they probably either formed in situ or formed farther out and then underwent disk migration. A comparison of NGTS-11 with the eight other known systems hosting multiple well-characterized warm giants shows that it is most similar to Kepler-56. Finally, we find that the commonly used 10-day boundary between hot and warm Jupiters is empirically well supported.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.14779-b31b1b.svg)](https://arxiv.org/abs/2510.14779) | **The dark side of early galaxies: $\texttt{geko}$ uncovers dark-matter fractions at $z\sim4-6$**  |
|| A. L. Danhaive, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-10-17*|
|*Comments*| *14 pages, 7 figures, 2 tables. Submitted to MNRAS*|
|**Abstract**|            JWST/NIRCam slitless spectroscopy enables dynamical mass measurements for typical star-forming galaxies only a billion years after the Big Bang. We model the H$\alpha$ morpho-kinematics of 163 galaxies at redshift $z\approx4$-6 from FRESCO and CONGRESS (with JADES imaging), using the $\texttt{geko}$ code, and infer rotational velocities and dispersions within $r_{\rm e}$. Our sample spans $\log M_{\star}\approx7$-10 and $\log M_{\rm dyn}\approx9$-11. Gas masses are estimated via scaling relations, yielding baryonic masses and dark-matter (DM) fractions $f_{\rm DM}(r<r_{\rm e})$ within the H$\alpha$ half-light radius. We find high median fractions of $\langle f_{\rm gas}\rangle=0.77$ and $\langle f_{\rm DM}\rangle=0.73$, where $f_{\rm gas}$ is measured with respect to the baryonic mass and $f_{\rm DM}$ with respect to the DM+baryonic mass. About two-thirds of systems are DM-dominated within $r_{\rm e}\sim0.5-1$ kpc. Both $f_{\rm gas}$ and $f_{\rm DM}$ decrease with stellar mass, consistent with simulations. The stellar Tully-Fisher relation shows a tentative offset to higher $v_{\rm circ}$ at fixed $M_{\star}$ and substantial intrinsic scatter, suggesting that the relation is only beginning to emerge at $z\sim5$. We measure a negative correlation between $f_{\rm DM}$ and baryonic surface density $\Sigma_{\rm bar}$, weaker but broadly consistent with trends at cosmic noon and at $z\sim0$. Qualitatively comparing with modified NFW profiles coupled to an empirical stellar-to-halo mass relation suggests that the lowest $f_{\rm DM}$ ($\lesssim0.4$) require cored inner DM profiles, while the highest fractions favour cuspier profiles, potentially reflecting adiabatic contraction. Overall, the elevated $f_{\rm gas}$ and $f_{\rm DM}$ at $z\gtrsim4$ are compatible with progenitors of baryon-dominated systems at $z\sim2$ and naturally anticipate overmassive black holes at fixed $M_{\star}$.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.13976-b31b1b.svg)](https://arxiv.org/abs/2510.13976) | **The ASTRID Simulation at z=0: from Massive Black Holes to Large-scale Structure**  |
|| Y. Zhou, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-17*|
|*Comments*| *Submitted to ApJ, comments are welcome*|
|**Abstract**|            We present the $z=0$ results for the cosmological simulation ASTRID. Hosting $2\times 5500^3\approx$ 0.33 trillion particles in a box of $370\, {\rm Mpc}$ per side, ASTRID is one of the largest cosmological hydrodynamic simulations evolved to $z=0$. ASTRID features a large population of massive black holes (MBHs), covering a wide mass range $4\times10^{4}\sim 2\times 10^{11}\ M_{\odot}$. The adopted dynamical friction model provides a relatively accurate description of MBH dynamics, making ASTRID a powerful tool to study MBH growth and mergers in a cosmological context. ASTRID successfully captures the co-evolution of MBHs and their host galaxies, producing $M_{\rm BH}-M_{\star}$ and $M_{\rm BH}-\sigma$ relations in good agreement with observations. Notably, ASTRID generates scatter in these relations that is more consistent with observations than previous simulations, indicating a more realistic MBH diversity. The galaxy stellar mass function at $z=0$ is generally consistent with observational constraints. When dust attenuation is applied, the galaxy luminosity function also agrees well with observations, and the bimodality in galaxy colors is reproduced as well. ASTRID hosts a large population of massive galaxy groups and clusters: 7 halos have $M_{\rm 200c}>10^{15}\ M_{\odot}$, and 9709 halos have $M_{\rm 200c}>10^{13}\ M_{\odot}$. We quantify the stellar mass content in these halos, and find that the correlations between the stellar and halo mass match well with observational constraints. Finally, we present the $z=0$ power spectra of MBH and galaxies, as well as their bias with respect to the matter power spectrum. We find that MBHs with $M_{\rm BH}\geq 10^{8}\ M_{\odot}$ and galaxies with $M_{\star}\geq 10^{10.5}\ M_{\odot}$ serve as good tracers of large-scale structure.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.14529-b31b1b.svg)](https://arxiv.org/abs/2510.14529) | **The variability of active galaxies: I. Broad-band noise X-ray power spectra from XMM-Newton and Swift**  |
|| M. Lefkir, et al. -- incl., <mark>S. Vaughan</mark> |
|*Appeared on*| *2025-10-17*|
|*Comments*| *22 pages, 10 figures, under review at MNRAS (after the first round of referee comments)*|
|**Abstract**|            Accreting supermassive black holes at the centres of galaxies are the engine of active galactic nuclei (AGN). X-ray light curves of unabsorbed AGN show dramatic random variability on timescales ranging from seconds to years. The power spectrum of the fluctuations is usually well-modelled with a power law that decays as $1/f$ at low frequencies, and which bends to $1/f^{2-3}$ at high frequencies. The timescale associated with the bend correlates well with the mass of the black hole and may also correlate with bolometric luminosity in the `X-ray variability plane'. Because AGN light curves are usually irregularly sampled, the estimation of AGN power spectra is challenging. In a previous paper, we introduced a new method to estimate the parameters of bending power law power spectra from AGN light curves. We apply this method to a sample of 56 variable and unabsorbed AGN, observed with XMM-Newton and Swift in the $0.3-1.5$ keV band over the past two decades. We obtain estimates of the bends in 50 sources, which is the largest sample of X-ray bends in the soft band. We also find that the high-frequency power spectrum is often steeper than 2. We update the X-ray variability plane with new bend timescale measurements spanning from 7 min to 62 days. We report the detections of low-frequency bends in the power spectra of five AGN, three of which are previously unpublished: 1H 1934-063, Mkn 766 and Mkn 279.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.14839-b31b1b.svg)](https://arxiv.org/abs/2510.14839) | **Antarctic Infrared Binocular Telescope. I. System Overview, Laboratory Testing, and On-Sky Performance Evaluation**  |
|| Z. Dong, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-10-17*|
|*Comments*| **|
|**Abstract**|            Infrared time-domain surveys remain significantly underdeveloped compared with their optical counterparts. We have developed the Antarctic Infrared Binocular Telescope (AIRBT) to study the dynamic infrared sky at Dome A, Antarctica, taking advantage of the superb infrared observational conditions at this site. AIRBT consists of two identical 15 cm f/3 optical tube assemblies and two cost-effective indium gallium arsenide (InGaAs) cameras equipped with J and H filters, respectively. The cameras have 640 x 512 pixels with a size of 15 micrometers, providing a scale of 6.9 arcseconds per pixel and a field of view of 1.22 x 0.97 square degrees. We characterize the performance of the InGaAs cameras, including bias, readout noise, dark current, nonlinearity, and photon transfer curve. Our analysis highlights the distinct behaviors of InGaAs cameras compared with charge-coupled devices (CCDs). The bias and readout noise show temperature dependence, and the noise measured from the photon transfer curves has additional components that increase with exposure time. On-sky tests were conducted in October 2022 including system calibration, limiting depth, and photometric precision. For a single 3-second exposure, we achieved 5-sigma limiting magnitudes of 11.2 mag (Vega system) in J band and 9.7 mag in H band. The best photometric precision reached 20 millimagnitudes at the bright end, which could be further improved to sub-percent levels through image stacking. AIRBT was installed at Dome A in January 2023, and scientific observations began as soon as darkness set in.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.14380-b31b1b.svg)](https://arxiv.org/abs/2510.14380) | **Dust extinction map of the Galactic plane based on the UKIDSS survey data**  |
|| <mark>M. Zhang</mark>, et al. |
|*Appeared on*| *2025-10-17*|
|*Comments*| *Accepted by MNRAS, 18 pages, 12 figures*|
|**Abstract**|            Dust plays a critical role in the study of the interstellar medium (ISM). Extinction maps derived from optical surveys often fail to capture regions with high column density due to the limited photometric depth in optical wavelengths. To address these limitations, we developed the XPNICER method based on near-infrared (NIR) photometric survey data. This method combines the previously established PNICER and Xpercentile techniques, enabling effective mitigation of foreground contamination and improved handling of complex dust structures in the Galactic plane, which thus can provide more accurate extinction estimates, particularly in highly obscured regions. By applying XPNICER to the Galactic Plane Survey from the UKIRT Infrared Deep Sky Survey, we have generated a series of two-dimensional (2D) dust extinction maps that span roughly 1800 deg2 of the Galactic plane (0< l < 110deg and 140< l < 232deg, |b| < 5deg). These maps, with spatial resolutions between 30arcsec and 300arcsec, can trace extinction up to Av ~ 30-40 mag. This new approach offers higher spatial resolution and better detection of high-extinction regions compared to previous large-scale dust-based maps of the Galactic plane, providing an independent and complementary measure of dust column densities.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.14083/./fig-mass-radius.png', 'tmp_2510.14083/./fig-mass-radius.png', 'tmp_2510.14083/./ngts11c.png', 'tmp_2510.14083/./ngts11b.png']
copying  tmp_2510.14083/./fig-mass-radius.png to _build/html/
copying  tmp_2510.14083/./fig-mass-radius.png to _build/html/
copying  tmp_2510.14083/./ngts11c.png to _build/html/
copying  tmp_2510.14083/./ngts11b.png to _build/html/
exported in  _build/html/2510.14083.md
    + _build/html/tmp_2510.14083/./fig-mass-radius.png
    + _build/html/tmp_2510.14083/./fig-mass-radius.png
    + _build/html/tmp_2510.14083/./ngts11c.png
    + _build/html/tmp_2510.14083/./ngts11b.png
found figures ['tmp_2510.14779/./Figures/fgas_fdm_Tacconi_zcolor.png', 'tmp_2510.14779/./Figures/fdm_sigmabar_multi.png', 'tmp_2510.14779/./Figures/alpha_fdm_mstar.png']
copying  tmp_2510.14779/./Figures/fgas_fdm_Tacconi_zcolor.png to _build/html/
copying  tmp_2510.14779/./Figures/fdm_sigmabar_multi.png to _build/html/
copying  tmp_2510.14779/./Figures/alpha_fdm_m

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\nbs}{\nobreak\hspace{0.15em}}$
$\newcommand{\feh}{\ensuremath{{\rm[Fe/H]}}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\zaspe}{\texttt{ZASPE}}$
$\newcommand{\ceres}{\texttt{CERES}}$
$\newcommand{\juliet}{{\sc juliet}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}}$
$\newcommand{\kms}{\ensuremath{{\rm km s^{-1}}}}$
$\newcommand{\ms}{\ensuremath{{\rm m s^{-1}}}}$
$\newcommand{\mjup}{\ensuremath{{M_{\rm Jup}}}}$
$\newcommand{\mnep}{\ensuremath{{M_{\rm Nep}}}}$
$\newcommand{\msat}{\ensuremath{{M_{\rm Sat}}}}$
$\newcommand{\mearth}{\ensuremath{M_{\rm \oplus}}}$
$\newcommand{\mpl}{\ensuremath{{M_{\rm pl}}}}$
$\newcommand{\rjup}{\ensuremath{R_{\rm Jup}}}$
$\newcommand{\rnep}{\ensuremath{R_{\rm Nep}}}$
$\newcommand{\rsat}{\ensuremath{R_{\rm Sat}}}$
$\newcommand{\rpl}{\ensuremath{R_{\rm pl}}}$
$\newcommand{\rstar}{\ensuremath{R_{\rm \star}}}$
$\newcommand{\mstar}{\ensuremath{M_{\rm \star}}}$
$\newcommand{\lstar}{\ensuremath{L_{\rm \star}}}$
$\newcommand{\rsun}{\ensuremath{R_{\rm \odot}}}$
$\newcommand{\msun}{\ensuremath{M_{\rm \odot}}}$
$\newcommand{\lsun}{\ensuremath{L_{\rm \odot}}}$
$\newcommand{\rhopl}{\ensuremath{{\rho_{\rm pl}}}}$
$\newcommand{\gccm}{\ensuremath{\rm{g} \rm{cm}^{-3}}}$
$\newcommand{\stnameA}{NGTS-11}$
$\newcommand{\plnameA}{NGTS-11\nbs c}$
$\newcommand{\plnameB}{NGTS-11\nbs b}$
$\newcommand{\plnameC}{NGTS-11\nbs d}$
$\newcommand{\altstnameA}{TOI-1847}$
$\newcommand{\altplnameA}{TOI-1847\nbs c}$
$\newcommand{\altplnameB}{TOI-1847\nbs b}$
$\newcommand{\altplnameC}{TOI-1847\nbs d}$
$\newcommand{\PA}{\ensuremath{12.77264 \pm 0.000017}}$
$\newcommand{\tcA}{\ensuremath{2459117.83838 \pm 0.00058}}$
$\newcommand{\pA}{\ensuremath{0.05297 \pm 0.00065}}$
$\newcommand{\bA}{\ensuremath{0.808 \pm 0.015}}$
$\newcommand{\KA}{\ensuremath{6.4^{+1.6}_{-1.5}}}$
$\newcommand{\PB}{\ensuremath{35.455882 \pm 0.000039}}$
$\newcommand{\tcB}{\ensuremath{2459135.28040 \pm 0.00055}}$
$\newcommand{\pB}{\ensuremath{0.1005 \pm 0.0011}}$
$\newcommand{\bB}{\ensuremath{0.846 \pm 0.014}}$
$\newcommand{\KB}{\ensuremath{12.9^{+1.7}_{-1.8}}}$
$\newcommand{\rhostA}{\ensuremath{1920 \pm 170}}$
$\newcommand{\qaTESSTRAPPIST}{\ensuremath{0.11 \pm 0.09}}$
$\newcommand{\qbTESSTRAPPIST}{\ensuremath{0.41 \pm 0.31}}$
$\newcommand{\qaNGTS}{\ensuremath{0.26 \pm 0.13}}$
$\newcommand{\qbNGTS}{\ensuremath{0.18 \pm 0.15}}$
$\newcommand{\qaMEarthULMTPEST}{\ensuremath{0.28 \pm 0.17}}$
$\newcommand{\qaLCOKeplerCamSAAO}{\ensuremath{0.25 \pm 0.11}}$
$\newcommand{\mfluxTESSA}{\ensuremath{-0.00002	\pm	0.00005}}$
$\newcommand{\mfluxTESSB}{\ensuremath{-0.00003	\pm	0.00005}}$
$\newcommand{\mfluxNGTS}{\ensuremath{0.00002	\pm	0.00003}}$
$\newcommand{\mfluxMEarth}{\ensuremath{-0.00002	\pm	0.00008}}$
$\newcommand{\mfluxLCO}{\ensuremath{-0.00006	\pm	0.00006}}$
$\newcommand{\mfluxKeplerCam}{\ensuremath{0.0000	\pm	0.0002}}$
$\newcommand{\mfluxULMT}{\ensuremath{-0.0001	\pm	0.0003}}$
$\newcommand{\mfluxTRAPPIST}{\ensuremath{-0.0001	\pm	0.0002}}$
$\newcommand{\mfluxPEST}{\ensuremath{-0.0001	\pm	0.0004}}$
$\newcommand{\mfluxSPECULOOS}{\ensuremath{0.0003	\pm	0.0002}}$
$\newcommand{\sigmaTESSA}{\ensuremath{{0	^{+	10	}_{-	0	}}}}$
$\newcommand{\sigmaTESSB}{\ensuremath{{2	^{+	150	}_{-	2	}}}}$
$\newcommand{\sigmaNGTS}{\ensuremath{773 \pm 36}}$
$\newcommand{\sigmaMEarth}{\ensuremath{{972 \pm 86}}}$
$\newcommand{\sigmaLCO}{\ensuremath{832 \pm 54	}}$
$\newcommand{\sigmaKeplerCam}{\ensuremath{2660 \pm 120 }}$
$\newcommand{\sigmaULMT}{\ensuremath{{0	^{+	39	}_{-	0	}}}}$
$\newcommand{\sigmaTRAPPIST}{\ensuremath{{2070 \pm 170 }}}$
$\newcommand{\sigmaPEST}{\ensuremath{0	^{+	45	}_{-	0	}}}$
$\newcommand{\sigmaSPECULOOS}{\ensuremath{0	^{+	45	}_{-	0	}}}$
$\newcommand{\sigmaSAAO}{\ensuremath{1	^{+	51	}_{-	0	}}}$
$\newcommand{\muFEROSA}{\ensuremath{21 384.0 \pm 8.2}}$
$\newcommand{\sigmawFEROSA}{\ensuremath{24.4 \pm 5.6}}$
$\newcommand{\muHARPSA}{\ensuremath{21 400.6 \pm 2.0}}$
$\newcommand{\dmuA}{\ensuremath{0.072 \pm 0.012}}$
$\newcommand{\ddmuA}{\ensuremath{0.000170 \pm 0.000019}}$
$\newcommand{\mpA}{\ensuremath{0.064^{+0.019}_{-0.013} }}$
$\newcommand{\mpAnep}{\ensuremath{1.2^{+0.3}_{-0.2}}}$
$\newcommand{\mpAearth}{\ensuremath{20^{+6}_{-4}}}$
$\newcommand{\rpA}{\ensuremath{0.429 \pm 0.009}}$
$\newcommand{\rpAnep}{\ensuremath{1.24 \pm 0.03}}$
$\newcommand{\rhoA}{\ensuremath{1040 \pm 260}}$
$\newcommand{\arstarA}{\ensuremath{25.5 \pm 0.7}}$
$\newcommand{\smaA}{\ensuremath{0.099 \pm 0.003}}$
$\newcommand{\smaAsimp}{\ensuremath{0.099}}$
$\newcommand{\incA}{\ensuremath{88.21 \pm 0.09}}$
$\newcommand{\tqA}{\ensuremath{594 \pm 13}}$
$\newcommand{\eccA}{\ensuremath{0.09}^{+0.09}_{-0.06}}$
$\newcommand{\omegaA}{\ensuremath{108^{+35}_{-49}}}$
$\newcommand{\mpB}{\ensuremath{0.189 \pm 0.026}}$
$\newcommand{\mpBsat}{\ensuremath{0.63 \pm 0.09}}$
$\newcommand{\mpBsatOld}{\ensuremath{1.2 \pm 0.3}}$
$\newcommand{\mpBearth}{\ensuremath{60 \pm 8}}$
$\newcommand{\rpB}{\ensuremath{0.81 \pm 0.02}}$
$\newcommand{\rpBsat}{\ensuremath{0.97 \pm 0.02}}$
$\newcommand{\rhoB}{\ensuremath{423 \pm 65}}$
$\newcommand{\arstarB}{\ensuremath{50 \pm 2}}$
$\newcommand{\smaB}{\ensuremath{0.195 \pm 0.006}}$
$\newcommand{\smaBsimp}{\ensuremath{0.195}}$
$\newcommand{\incB}{\ensuremath{89.05 \pm 0.04}}$
$\newcommand{\tqB}{\ensuremath{423 \pm 9}}$
$\newcommand{\eccB}{\ensuremath{0.08^{+0.06}_{-0.05}}}$
$\newcommand{\omegaB}{\ensuremath{12^{+56}_{-10}}}$
$\newcommand{\nbs}{\nobreak\hspace{0.15em}}$
$\newcommand{\feh}{\ensuremath{{\rm[Fe/H]}}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\zaspe}{\texttt{ZASPE}}$
$\newcommand{\ceres}{\texttt{CERES}}$
$\newcommand{\juliet}{{\sc juliet}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}}$
$\newcommand{\kms}{\ensuremath{{\rm km s^{-1}}}}$
$\newcommand{\ms}{\ensuremath{{\rm m s^{-1}}}}$
$\newcommand{\mjup}{\ensuremath{{M_{\rm Jup}}}}$
$\newcommand{\mnep}{\ensuremath{{M_{\rm Nep}}}}$
$\newcommand{\msat}{\ensuremath{{M_{\rm Sat}}}}$
$\newcommand{\mearth}{\ensuremath{M_{\rm \oplus}}}$
$\newcommand{\mpl}{\ensuremath{{M_{\rm pl}}}}$
$\newcommand{\rjup}{\ensuremath{R_{\rm Jup}}}$
$\newcommand{\rnep}{\ensuremath{R_{\rm Nep}}}$
$\newcommand{\rsat}{\ensuremath{R_{\rm Sat}}}$
$\newcommand{\rpl}{\ensuremath{R_{\rm pl}}}$
$\newcommand{\rstar}{\ensuremath{R_{\rm \star}}}$
$\newcommand{\mstar}{\ensuremath{M_{\rm \star}}}$
$\newcommand{\lstar}{\ensuremath{L_{\rm \star}}}$
$\newcommand{\rsun}{\ensuremath{R_{\rm \odot}}}$
$\newcommand{\msun}{\ensuremath{M_{\rm \odot}}}$
$\newcommand{\lsun}{\ensuremath{L_{\rm \odot}}}$
$\newcommand{\rhopl}{\ensuremath{{\rho_{\rm pl}}}}$
$\newcommand{\gccm}{\ensuremath{\rm{g} \rm{cm}^{-3}}}$
$\newcommand{\stnameA}{NGTS-11}$
$\newcommand{\plnameA}{NGTS-11\nbs c}$
$\newcommand{\plnameB}{NGTS-11\nbs b}$
$\newcommand{\plnameC}{NGTS-11\nbs d}$
$\newcommand{\altstnameA}{TOI-1847}$
$\newcommand{\altplnameA}{TOI-1847\nbs c}$
$\newcommand{\altplnameB}{TOI-1847\nbs b}$
$\newcommand{\altplnameC}{TOI-1847\nbs d}$
$\newcommand{\PA}{\ensuremath{12.77264 \pm 0.000017}}$
$\newcommand{\tcA}{\ensuremath{2459117.83838 \pm 0.00058}}$
$\newcommand{\pA}{\ensuremath{0.05297 \pm 0.00065}}$
$\newcommand{\bA}{\ensuremath{0.808 \pm 0.015}}$
$\newcommand{\KA}{\ensuremath{6.4^{+1.6}_{-1.5}}}$
$\newcommand{\PB}{\ensuremath{35.455882 \pm 0.000039}}$
$\newcommand{\tcB}{\ensuremath{2459135.28040 \pm 0.00055}}$
$\newcommand{\pB}{\ensuremath{0.1005 \pm 0.0011}}$
$\newcommand{\bB}{\ensuremath{0.846 \pm 0.014}}$
$\newcommand{\KB}{\ensuremath{12.9^{+1.7}_{-1.8}}}$
$\newcommand{\rhostA}{\ensuremath{1920 \pm 170}}$
$\newcommand{\qaTESSTRAPPIST}{\ensuremath{0.11 \pm 0.09}}$
$\newcommand{\qbTESSTRAPPIST}{\ensuremath{0.41 \pm 0.31}}$
$\newcommand{\qaNGTS}{\ensuremath{0.26 \pm 0.13}}$
$\newcommand{\qbNGTS}{\ensuremath{0.18 \pm 0.15}}$
$\newcommand{\qaMEarthULMTPEST}{\ensuremath{0.28 \pm 0.17}}$
$\newcommand{\qaLCOKeplerCamSAAO}{\ensuremath{0.25 \pm 0.11}}$
$\newcommand{\mfluxTESSA}{\ensuremath{-0.00002	\pm	0.00005}}$
$\newcommand{\mfluxTESSB}{\ensuremath{-0.00003	\pm	0.00005}}$
$\newcommand{\mfluxNGTS}{\ensuremath{0.00002	\pm	0.00003}}$
$\newcommand{\mfluxMEarth}{\ensuremath{-0.00002	\pm	0.00008}}$
$\newcommand{\mfluxLCO}{\ensuremath{-0.00006	\pm	0.00006}}$
$\newcommand{\mfluxKeplerCam}{\ensuremath{0.0000	\pm	0.0002}}$
$\newcommand{\mfluxULMT}{\ensuremath{-0.0001	\pm	0.0003}}$
$\newcommand{\mfluxTRAPPIST}{\ensuremath{-0.0001	\pm	0.0002}}$
$\newcommand{\mfluxPEST}{\ensuremath{-0.0001	\pm	0.0004}}$
$\newcommand{\mfluxSPECULOOS}{\ensuremath{0.0003	\pm	0.0002}}$
$\newcommand{\sigmaTESSA}{\ensuremath{{0	^{+	10	}_{-	0	}}}}$
$\newcommand{\sigmaTESSB}{\ensuremath{{2	^{+	150	}_{-	2	}}}}$
$\newcommand{\sigmaNGTS}{\ensuremath{773 \pm 36}}$
$\newcommand{\sigmaMEarth}{\ensuremath{{972 \pm 86}}}$
$\newcommand{\sigmaLCO}{\ensuremath{832 \pm 54	}}$
$\newcommand{\sigmaKeplerCam}{\ensuremath{2660 \pm 120 }}$
$\newcommand{\sigmaULMT}{\ensuremath{{0	^{+	39	}_{-	0	}}}}$
$\newcommand{\sigmaTRAPPIST}{\ensuremath{{2070 \pm 170 }}}$
$\newcommand{\sigmaPEST}{\ensuremath{0	^{+	45	}_{-	0	}}}$
$\newcommand{\sigmaSPECULOOS}{\ensuremath{0	^{+	45	}_{-	0	}}}$
$\newcommand{\sigmaSAAO}{\ensuremath{1	^{+	51	}_{-	0	}}}$
$\newcommand{\muFEROSA}{\ensuremath{21 384.0 \pm 8.2}}$
$\newcommand{\sigmawFEROSA}{\ensuremath{24.4 \pm 5.6}}$
$\newcommand{\muHARPSA}{\ensuremath{21 400.6 \pm 2.0}}$
$\newcommand{\dmuA}{\ensuremath{0.072 \pm 0.012}}$
$\newcommand{\ddmuA}{\ensuremath{0.000170 \pm 0.000019}}$
$\newcommand{\mpA}{\ensuremath{0.064^{+0.019}_{-0.013} }}$
$\newcommand{\mpAnep}{\ensuremath{1.2^{+0.3}_{-0.2}}}$
$\newcommand{\mpAearth}{\ensuremath{20^{+6}_{-4}}}$
$\newcommand{\rpA}{\ensuremath{0.429 \pm 0.009}}$
$\newcommand{\rpAnep}{\ensuremath{1.24 \pm 0.03}}$
$\newcommand{\rhoA}{\ensuremath{1040 \pm 260}}$
$\newcommand{\arstarA}{\ensuremath{25.5 \pm 0.7}}$
$\newcommand{\smaA}{\ensuremath{0.099 \pm 0.003}}$
$\newcommand{\smaAsimp}{\ensuremath{0.099}}$
$\newcommand{\incA}{\ensuremath{88.21 \pm 0.09}}$
$\newcommand{\tqA}{\ensuremath{594 \pm 13}}$
$\newcommand{\eccA}{\ensuremath{0.09}^{+0.09}_{-0.06}}$
$\newcommand{\omegaA}{\ensuremath{108^{+35}_{-49}}}$
$\newcommand{\mpB}{\ensuremath{0.189 \pm 0.026}}$
$\newcommand{\mpBsat}{\ensuremath{0.63 \pm 0.09}}$
$\newcommand{\mpBsatOld}{\ensuremath{1.2 \pm 0.3}}$
$\newcommand{\mpBearth}{\ensuremath{60 \pm 8}}$
$\newcommand{\rpB}{\ensuremath{0.81 \pm 0.02}}$
$\newcommand{\rpBsat}{\ensuremath{0.97 \pm 0.02}}$
$\newcommand{\rhoB}{\ensuremath{423 \pm 65}}$
$\newcommand{\arstarB}{\ensuremath{50 \pm 2}}$
$\newcommand{\smaB}{\ensuremath{0.195 \pm 0.006}}$
$\newcommand{\smaBsimp}{\ensuremath{0.195}}$
$\newcommand{\incB}{\ensuremath{89.05 \pm 0.04}}$
$\newcommand{\tqB}{\ensuremath{423 \pm 9}}$
$\newcommand{\eccB}{\ensuremath{0.08^{+0.06}_{-0.05}}}$
$\newcommand{\omegaB}{\ensuremath{12^{+56}_{-10}}}$</div>



<div id="title">

# $\plnameA$: a transiting Neptune-mass planet interior to the warm Saturn $\plnameB$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.14083-b31b1b.svg)](https://arxiv.org/abs/2510.14083)<mark>Appeared on: 2025-10-17</mark> -  _12 pages, 6 figures, submitted to ApJ_

</div>
<div id="authors">

D. R. Anderson, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of $\plnameA$ , a transiting warm Neptune ( $P \sim 12.8$ d; $\mpl = \mpAnep \mnep$ ; $\rpl = \rpAnep \rnep$ ),orbiting closer to the K-dwarf $\stnameA$ than the previously reported transiting warm Saturn $\plnameB$ ( $P \sim 35.5$ $\nbs$ d), and we report evidence of a third, outer companion.We first detected transits of $\plnameA$ in TESS light curves and confirmed them with follow-up transits from NGTS and many other ground-based facilities.Radial-velocity monitoring with the HARPS and FEROS spectrographs revealed the mass of $\plnameA$ and provided evidence, via a long-term trend, for a long-period companion ( $P > 2300$ $\nbs$ d and $\mpl \sin i > 3.6$ $\nbs$ $\mjup$ ).Taking into account the two additional bodies in our expanded data sets, we find that the mass of $\plnameB$ ( $\mpl$ = $\mpBsat$ $\msat$ ; $\rpl$ = $\rpBsat$ $\rsat$ ) is lower than previously reported ( $\mpl$ = $\mpBsatOld$ $\msat$ ).Given their near-circular, compact orbits, $\plnameA$ and b are unlikely to have arrived in place via high-eccentricity migration.Instead, they likely formed in situ, or they formed farther out and then moved inward through disk migration.A comparison of $\stnameA$ with the eight other known systems hosting multiple well-characterized warm giants reveals that it is most similar to Kepler-56.Finally, we find that the commonly used 10-day boundary between hot and warm Jupiters appears empirically well supported.

</div>

<div id="div_fig1">

<img src="tmp_2510.14083/./fig-mass-radius.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
Mass--radius distribution of known exoplanets (gray dots have $P<10$\nbs d and orange triangles have 10$\nbs$$\le P \le 200$\nbs d), solar system giants (blue circles), and $\plnameA$ and b (brown diamonds, with error bars).
At $\mpl$$\sim 0.2$\nbs$\mjup$, $\plnameB$ approximately marks the mass boundary above which hot Jupiters are often inflated, whereas warm Jupiters generally are not.
Data were retrieved on 2025 Oct 1 from the composite data table of the NASA Exoplanet Archive.
Planets with masses inferred from a mass--radius relationship were excluded, as were those with imprecise parameters ($\Delta$\mpl/$\mpl$$>$ 0.5 or $\Delta$\rpl/$\rpl$$>$ 0.2).
 (*fig:m-r*)

</div>
<div id="div_fig2">

<img src="tmp_2510.14083/./fig-mass-radius.png" alt="Fig5" width="100%"/>

**Figure 5. -** 
Mass--radius distribution of known exoplanets (gray dots have $P<10$\nbs d and orange triangles have 10$\nbs$$\le P \le 200$\nbs d), solar system giants (blue circles), and $\plnameA$ and b (brown diamonds, with error bars).
At $\mpl$$\sim 0.2$\nbs$\mjup$, $\plnameB$ approximately marks the mass boundary above which hot Jupiters are often inflated, whereas warm Jupiters generally are not.
Data were retrieved on 2025 Oct 1 from the composite data table of the NASA Exoplanet Archive.
Planets with masses inferred from a mass--radius relationship were excluded, as were those with imprecise parameters ($\Delta$\mpl/$\mpl$$>$ 0.5 or $\Delta$\rpl/$\rpl$$>$ 0.2).
 (*fig:m-r*)

</div>
<div id="div_fig3">

<img src="tmp_2510.14083/./ngts11c.png" alt="Fig7.1" width="50%"/><img src="tmp_2510.14083/./ngts11b.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Transit light curves of $\plnameA$(left panel) and $\plnameB$(right panel), shown offset in relative flux for clarity and plotted to the same scale for comparison.
The data were phase-binned to 10$\nbs$ min, except for TESS Sector 3 (30$\nbs$ min) and TRAPPIST and PEST (20$\nbs$ min for both).
The error bars represent formal uncertainties and do not include the photometric jitter terms (\autoref{tab:props}).
The transit model (\autoref{sec:anal}), including limb darkening, is shown for each case.
 (*fig:phot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.14083"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kthrd}{\ensuremath{k_{\mathrm{3D}}}\xspace}$
$\newcommand{\bn}{\ensuremath{b_{\mathrm{n}}}\xspace}$
$\newcommand{\Reff}{\ensuremath{r_{\mathrm{e}}}\xspace}$
$\newcommand{\D}{\ensuremath{\mathrm{d}}\xspace}$
$\newcommand{\qint}{\ensuremath{q_0}\xspace}$
$\newcommand{\qobs}{\ensuremath{q_{\mathrm{obs}}}\xspace}$
$\newcommand{\vcirc}{\ensuremath{v_{\mathrm{circ}}}\xspace}$
$\newcommand{\vcircobs}{\ensuremath{v_{\mathrm{circ,obs}}}\xspace}$
$\newcommand{\Menc}{\ensuremath{M_{\mathrm{sph}}}\xspace}$
$\newcommand{\vrot}{\ensuremath{v_{\mathrm{rot}}}\xspace}$
$\newcommand{\sigmaint}{\ensuremath{\sigma_{0}}\xspace}$
$\newcommand{\Msun}{\ensuremath{\rm M_{\odot}}\xspace}$
$\newcommand{\Mstar}{\ensuremath{M_{\star}}\xspace}$
$\newcommand{\Mgas}{\ensuremath{M_{\mathrm{gas}}}\xspace}$
$\newcommand{\Mbar}{\ensuremath{M_{\mathrm{bar}}}\xspace}$
$\newcommand{\Mdyn}{\ensuremath{M_{\mathrm{dyn}}}\xspace}$
$\newcommand{\Mbh}{\ensuremath{M_{\mathrm{BH}}}\xspace}$
$\newcommand{\lMstar}{\ensuremath{\log_{10}(\Mstar/\Msun)}\xspace}$
$\newcommand{\lMgas}{\ensuremath{\log_{10}(\Mgas/\Msun)}\xspace}$
$\newcommand{\lMbar}{\ensuremath{\log_{10}(\Mbar/\Msun)}\xspace}$
$\newcommand{\lMdyn}{\ensuremath{\log_{10}(\Mdyn/\Msun)}\xspace}$
$\newcommand{\logMstar}{\ensuremath{\log(\Mstar ~\rm[\Msun])}\xspace}$
$\newcommand{\logMbh}{\ensuremath{\log(\Mbh ~\rm[\Msun])}\xspace}$
$\newcommand{\logMdyn}{\ensuremath{\log(\Mdyn ~\rm[\Msun])}\xspace}$
$\newcommand{\fgas}{\ensuremath{f_{\mathrm{gas}}}\xspace}$
$\newcommand{\fDM}{\ensuremath{f_{\mathrm{DM}}}\xspace}$
$\newcommand{\nSdisk}{\ensuremath{n_{\mathrm{disk}}}\xspace}$
$\newcommand{\nSbulge}{\ensuremath{n_{\mathrm{bulge}}}\xspace}$
$\newcommand{\Ha}{\ensuremath{\mathrm{H}\alpha}\xspace}$
$\newcommand{\vsigma}{\ensuremath{\mathrm{v}/\sigmaint}\xspace}$
$\newcommand{\geko}{\textsc{geko}}$
$\newcommand{\sersic}{Sérsic }$
$\newcommand{\angstrom}{\mbox{\normalfontÅ}}$
$\newcommand{\deltaMS}{\ensuremath{\Delta \rm MS}\xspace}$
$\newcommand{\rotsupp}{\text{v}/\sigma_0}$
$\newcommand{\disp}{\sigma_0}$
$\newcommand{\rHa}{\ensuremath{r_{\rm e, Ha}}\xspace}$
$\newcommand{\rNUV}{\ensuremath{r_{\rm e, NUV}}\xspace}$
$\newcommand{\rFUV}{\ensuremath{r_{\rm e, FUV}}\xspace}$
$\newcommand{\ropt}{\ensuremath{r_{\rm e, opt}}\xspace}$
$\newcommand{\HatoUV}{\ensuremath{r_{\rm e, Ha}/r_{\rm e, UV}}\xspace}$
$\newcommand{\HatoNUV}{\ensuremath{r_{\rm e, Ha}/r_{\rm e, NUV}}\xspace}$
$\newcommand{\FUVtoNUV}{\ensuremath{r_{\rm e, FUV}/r_{\rm e, NUV}}\xspace}$
$\newcommand{\FUVtoopt}{\ensuremath{r_{\rm e, FUV}/r_{\rm e, opt}}\xspace}$
$\newcommand{\sigmabar}{\ensuremath{\Sigma_{\rm bar}}\xspace}$
$\newcommand{\logsigmabar}{\ensuremath{\log (\Sigma_{\rm bar}) [\Msun \thinspace \rm kpc^{-2}]}\xspace}$
$\newcommand$
$\newcommand{\labelitemi}{\labelitemii}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The dark side of early galaxies: $\geko$ uncovers dark-matter fractions at$z\sim4-6$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.14779-b31b1b.svg)](https://arxiv.org/abs/2510.14779)<mark>Appeared on: 2025-10-17</mark> -  _14 pages, 7 figures, 2 tables. Submitted to MNRAS_

</div>
<div id="authors">

A. L. Danhaive, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** _JWST_ /NIRCam slitless spectroscopy enables dynamical mass measurements for typical star-forming galaxies only a billion years after the Big Bang. We model the H $\alpha$ morpho-kinematics of 163 galaxies at redshift $z\approx4$ -6 from FRESCO and CONGRESS (with JADES imaging), using the $\texttt{geko}$ code, and infer rotational velocities and dispersions within $r_{\rm e}$ . Our sample spans $\log M_{\star}\approx7$ -10 and $\log M_{\rm dyn}\approx9$ -11. Gas masses are estimated via scaling relations, yielding baryonic masses and dark-matter (DM) fractions $f_{\rm DM}(r<r_{\rm e})$ within the H $\alpha$ half-light radius. We find high median fractions of $\langle f_{\rm gas}\rangle=0.77$ and $\langle f_{\rm DM}\rangle=0.73$ , where $f_{\rm gas}$ is measured with respect to the baryonic mass and $f_{\rm DM}$ with respect to the DM+baryonic mass. About two-thirds of systems are DM-dominated within $r_{\rm e}\sim0.5-1$ kpc. Both $f_{\rm gas}$ and $f_{\rm DM}$ decrease with stellar mass, consistent with simulations. The stellar Tully-Fisher relation shows a tentative offset to higher $v_{\rm circ}$ at fixed $M_{\star}$ and substantial intrinsic scatter, suggesting that the relation is only beginning to emerge at $z\sim5$ . We measure a negative correlation between $f_{\rm DM}$ and baryonic surface density $\Sigma_{\rm bar}$ , weaker but broadly consistent with trends at cosmic noon and at $z\sim0$ . Qualitatively comparing with modified NFW profiles coupled to an empirical stellar-to-halo mass relation suggests that the lowest $f_{\rm DM}$ ( $\lesssim0.4$ ) require cored inner DM profiles, while the highest fractions favour cuspier profiles, potentially reflecting adiabatic contraction. Overall, the elevated $f_{\rm gas}$ and $f_{\rm DM}$ at $z\gtrsim4$ are compatible with progenitors of baryon-dominated systems at $z\sim2$ and naturally anticipate overmassive black holes at fixed $M_{\star}$ .

</div>

<div id="div_fig1">

<img src="tmp_2510.14779/./Figures/fgas_fdm_Tacconi_zcolor.png" alt="Fig3" width="100%"/>

**Figure 3. -** Gas (top) and DM (bottom) fractions (within $\Reff$) as a function of stellar mass, with the characteristic uncertainty shown in grey. The $f_{\text{gas}} (r<$\Reff$)$(Eq. \ref{eq:fgas}) is computed using the sSFR, redshift, and stellar mass of each galaxy following \protect[Tacconi, Genzel and Sternberg (2020)](https://ui.adsabs.harvard.edu/abs/2020ARA&A..58..157T). The $f_{\text{DM}} (r<$\Reff$)$ is computed using $M_{\text{gas}}$ along with $M_{\text{dyn}}$ and $M_{\star}$ as shown in Eq. \ref{eq:fdm}. We fit our data with a power-law (solid purple line) for both relations, showing the intrinsic scatter (shaded region) only for $\fDM$. We cannot constrain $\sigma_{\rm int}$ for $\fgas$ due to the inferred uncertainties being significantly larger than the scatter in the outputs of the [Tacconi, Genzel and Sternberg (2020)](https://ui.adsabs.harvard.edu/abs/2020ARA&A..58..157T) scaling relation for our sample (see Sec. \ref{sec:gas-fracs}). We compare our results with measurements from [ and Genzel (2020)](https://ui.adsabs.harvard.edu/abs/2020ApJ...902...98G)(green stars) and Nestor-Shachar:2023aa(RC100; blue diamonds) at cosmic noon and de-Graaff:2024ab, [Pillepich and Rix (2024)](https://ui.adsabs.harvard.edu/abs/2024ApJ...967L..40D)(orange pentagons) and [ and Lee (2025)](https://ui.adsabs.harvard.edu/abs/2025arXiv250711600L)(pink hexagons) at high redshift. For most of our sample, we find high gas and DM fractions $f > 0.5$. We find relatively good agreement with median trends from the thesan-zoom simulations \citep[green solid line;][]{McClymont:2025ac} and the tng50 simulations \citep[blue solid lines;][]{de-Graaff:2024aa}. (*fig:fractions-mass*)

</div>
<div id="div_fig2">

<img src="tmp_2510.14779/./Figures/fdm_sigmabar_multi.png" alt="Fig6" width="100%"/>

**Figure 6. -** DM fraction as a function of baryonic surface density ($\Sigma_{\rm bar}$) for our sample (circles) and samples at lower \citep[diamonds, ][]{Nestor-Shachar:2023aa} and similar to higher \citep[pentagons, ][]{de-Graaff:2024aa} redshift. We colour-code points by stellar mass (left panel) and gas fractions (right panel). We show the characteristic uncertainty for our sample in grey. We find a strong negative correlation between $\fDM$ and $\Sigma_{\rm bar}$($\rho = -0.551, p < 0.001$), with a best-fit relation (solid purple line) steeper than [ and Wuyts (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...831..149W) at cosmic noon (dashed brown line), and with an offset. We also plot the best-fit relation obtain when fitting our points with those from Nestor-Shachar:2023aa(dashed-dotted red line). For comparison, we plot medians from ETGs at $z\sim 0$ from the ATLAS3D survey \citep[purple square; ][]{Cappellari:2013aa} and LTGs from the DiskMass Survey \citep[grey triangles; ][]{Martinsson:2013aa}. Our galaxies agree with Nestor-Shachar:2023aa at low fractions $\fDM<0.5$, but lie above the relation at high fractions. The galaxies in our sample are smaller, less massive, and more gas rich than those probed at cosmic noon, driving the apparent shift in the relation.  (*fig:fdm-sigmabar*)

</div>
<div id="div_fig3">

<img src="tmp_2510.14779/./Figures/alpha_fdm_mstar.png" alt="Fig7" width="100%"/>

**Figure 7. -** Estimated slope $\alpha$ of the inner DM density profile for a modified NFW distribution (Eq. \ref{eq:gen-nfw}) as a function of measured DM fractions (left) and stellar mass (right). A value of $\alpha = 1$ represents the classical NFW profile, whereas $\alpha < 1$ corresponds to a cored profile, and $\alpha>1$ to a cuspier profile. Under the assumed SHMR and concentration ($c=5$), we find that low-mass galaxies with high $\fDM$ are associated with cuspy distributions, converging towards an NFW-like profile at higher masses with a scatter at least in part driven by their star-formation state. To highlight this, we colour-code the galaxies by the ratio of their SFR averaged over 10 and 100 Myr, $\rm SFR_{10}/SFR_{100}$, applying LOESS smoothing  ([ and Cappellari 2013](https://ui.adsabs.harvard.edu/abs/2013MNRAS.432.1862C)) . Galaxies going through a burst of star formation ($\log \rm SFR_{10}/SFR_{100} \gtrapprox 0.4$) have preferentially high values of $\alpha$, which supports the adiabatic contraction scenario. (*fig:alpha_fdm_sfr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.14779"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

118  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
